In [1]:
!pip install -q langchain langchain-community langchain-openai faiss-cpu sentence-transformers pypdf

In [2]:
!pip install -q pypdf

In [3]:
!pip install -q langchain-text-splitters

In [4]:
!pip install -q \
langchain \
langchain-community \
langchain-openai \
langchain-text-splitters \
faiss-cpu \
sentence-transformers \
pypdf

In [5]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

print("Uploaded file:", file_name)

Saving final RScopyjobf(+++).pdf to final RScopyjobf(+++) (1).pdf
Uploaded file: final RScopyjobf(+++) (1).pdf


In [6]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader

file_extension = file_name.split(".")[-1].lower()

if file_extension == "txt":
    loader = TextLoader(file_name, encoding="utf-8")
elif file_extension == "pdf":
    loader = PyPDFLoader(file_name)
else:
    raise ValueError("Unsupported file type. Please upload .txt or .pdf")

documents = loader.load()

print("File loaded successfully!")

File loaded successfully!


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)

print("Number of chunks:", len(chunks))

Number of chunks: 12


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/tmp/ipython-input-9218/384917042.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [9]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embedding_model)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print("Vector database created successfully!")

Vector database created successfully!


In [10]:
import os

os.environ["OPENROUTER_API_KEY"] = "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"

In [11]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model="meta-llama/llama-3-8b-instruct",
    openai_api_key=os.environ["OPENROUTER_API_KEY"],
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0
)

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate(
    template="""
You are an assistant answering strictly from the context.

Rules:
- Use ONLY the context.
- If answer not found, say:
  "I don't have information about that in the provided file."

Context:
{context}

Question:
{question}

Answer:
""",
    input_variables=["context", "question"],
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
while True:
    query = input("\nAsk a question (or type 'exit'): ")

    if query.lower() == "exit":
        break

    answer = rag_chain.invoke(query)
    print("\nAnswer:", answer)


Ask a question (or type 'exit'): What is my name?

Answer: Aysha Mohamed Elsayed

Ask a question (or type 'exit'): What is my educational background?

Answer: According to the provided context, your educational background is:

* Bachelor's in Artificial Intelligence from Alamein International University (AIU), Egypt
* Graduated: June 2025 | CGPA: 3.371/4.0

Ask a question (or type 'exit'): What technical skills do I have?

Answer: According to the provided context, your technical skills are:

* Programming: Python, Java, C, R
* Machine Learning: TensorFlow, PyTorch, HuggingFace Transformers, Scikit-learn

Ask a question (or type 'exit'): Which AI frameworks have I worked with?

Answer: According to the provided context, you have worked with the following AI frameworks:

* LangChain
* Groq API
* HuggingFace Transformers

These are mentioned in your summary section as part of your experience designing scalable, multilingual NLP and vision systems.

Ask a question (or type 'exit'): Tell 